# Binary Answer Type

This notebook demonstrates how to generate forecasting questions with binary (yes/no) answers. Binary questions are the simplest and most common type, perfect for event prediction, outcome forecasting, and decision-making scenarios.

In [ ]:
%pip install -e ..
%pip install python-dotenv

from IPython.display import clear_output
clear_output()

## Set up the client

**Set your API key:**

- **Locally**: Create a `.env` file in this directory with `LIGHTNINGROD_API_KEY=your_key_here`
- **Google Colab**: Go to the Secrets section (🔑 icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`

In [ ]:
from utils import get_secret
from lightningrod import LightningRod

api_key = get_secret("LIGHTNINGROD_API_KEY")
base_url = get_secret("LIGHTNINGROD_BASE_URL", "https://api.lightningrod.ai/api/public/v1")

lr = LightningRod(api_key=api_key, base_url=base_url)

## Binary Answer Type Overview

Binary questions have two possible answers: Yes/No or True/False. They're ideal for:
- Event prediction ("Will X happen?")
- Outcome forecasting ("Will Y succeed?")
- Decision-making scenarios ("Should we do Z?")

Binary questions work with any data source: News Search, Top Aggregated News, or Custom Documents.

In [ ]:
from datetime import datetime, timedelta
from lightningrod import QuestionGenerator, WebSearchLabeler, QuestionRenderer, QuestionPipeline, AnswerType, AnswerTypeEnum, NewsSeedGenerator

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query="technology product launches",
)

question_generator = QuestionGenerator(
    instructions=(
        "Generate binary forecasting questions about technology product launches. "
        "Questions should be answerable with Yes or No."
    ),
    examples=[
        "Will Apple release a new iPhone this year?",
        "Will Google launch a new AI product in Q2?",
        "Will Microsoft release Windows 12 in 2025?",
    ],
    bad_examples=[
        "What features will the new iPhone have?",
        "When will Google launch the product?",
        "How much will Windows 12 cost?",
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

# Renderer formats the question output
renderer = QuestionRenderer(answer_type=answer_type)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

## Run the Pipeline

Generate binary questions from the news articles.

In [ ]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [ ]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [ ]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df

Generated 30 samples

                              question.question_text   label.label  \
0      Will Apple release a foldable iPhone in 2026?             1   
1  Will the FrostSkin Kickstarter campaign launch...             0   
2  Will Nvidia officially announce the GeForce RT...             0   
3  Will U.K. retail investors gain access to cryp...             1   
4  Will Apple Inc. announce a new hardware produc...  Undetermined   

   label.label_confidence label.resolution_date  \
0                     0.9   2026-01-19T00:00:00   
1                     0.9   2026-01-06T00:00:00   
2                     0.9   2026-01-05T00:00:00   
3                     1.0   2025-08-01T00:00:00   
4                     0.9                   NaN   

                                     label.reasoning  \
0  Multiple reputable sources and analyst reports...   
1  As of January 20, 2026, there is no informatio...   
2  Nvidia did not officially announce new GeForce...   
3  The Financial Conduct A

## When to Use Binary vs Other Answer Types

**Use Binary when:**
- You need simple yes/no answers
- Predicting events or outcomes
- Building decision-making systems
- You want the simplest evaluation

**Consider other types when:**
- You need numeric values → Use Continuous
- You need multiple options → Use Multiple Choice
- You need open-ended explanations → Use Free Response